# ROE preprocessor
The programm preprocesses US stocks income statements and balance sheets to calculate ROE. Then, the data about the most recent (publicly available, i.e., after the report date) ROE for each ticker for each month within the given range of years is saved to a CSV file. 

If the most recent Total Equity of a company is negative, the corresponding rows are not saved in order to avoid distorting the regression with misleading ROE values.

The data was downloaded from https://app.simfin.com

### 1. Imports and helper functions

In [23]:
import pandas as pd
import numpy as np

def precompute_ticker_data(data):
    grouped = {ticker: df for ticker, df in data.groupby('Ticker')}
    return grouped

def get_latest_roe(ticker, date, grouped_data):    
    ticker_data = grouped_data[ticker]
    report_dates = ticker_data['Report Date'].values
    equities = ticker_data['Total Equity'].values
    roes = ticker_data['Return on Equity'].values

    # Data is in ascending order by the report date
    i = 0
    n = len(ticker_data) 
    while (i < n and report_dates[i] < date ):
        i += 1
        
    if i == 0 or equities[i - 1] <= 0:
        return np.nan

    return roes[i - 1]


### 2. Main function

In [26]:
def calculate_roes(income_filename, balance_filename, start_year, end_year):
    income_data = pd.read_csv(income_filename, sep=';', encoding='utf-8-sig')
    balance_data = pd.read_csv(balance_filename, sep=';', encoding='utf-8-sig')
    
    print(f"Number of rows for income: {len(income_data)}")
    print(f"Number of rows for balance: {len(balance_data)}")
    
    merged_data = pd.merge(income_data, balance_data, on=['Ticker', 'Fiscal Year', 'Fiscal Period'], suffixes=('_income', '_balance'))
    print(f"Number of rows after merge: {len(merged_data)}")

    # Setting max of two report dates for the fiscal period as a ROE report date
    merged_data['Report Date'] = pd.to_datetime(merged_data[['Report Date_income', 'Report Date_balance']].max(axis=1))
    merged_data['Return on Equity'] = merged_data['Net Income'] * 100 / merged_data['Total Equity']

    # Creating dataset for every month for the given year period
    all_months = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='MS')
    tickers = merged_data['Ticker'].unique()
    index = pd.MultiIndex.from_product([tickers, all_months], names=['Ticker', 'Date'])
    monthly_data = pd.DataFrame(index=index).reset_index()
    
    monthly_data['Year'] = monthly_data['Date'].dt.year
    monthly_data['Month'] = monthly_data['Date'].dt.month
    
    grouped_data = precompute_ticker_data(merged_data) # Grouping data for every ticker in a dictionary, so searching for a ticker is faster
    monthly_data['Return on Equity'] = monthly_data.apply(lambda row: get_latest_roe(row['Ticker'], row['Date'], grouped_data), axis=1)

    monthly_data = monthly_data.dropna()
    print(f"Final number of rows: {len(monthly_data)}")
    
    return monthly_data[['Ticker', 'Year', 'Month', 'Return on Equity']]

### 3. Data and programm execution

In [27]:
income_filename = 'datasets/us-income-ttm.csv'
balance_filename = 'datasets/us-balance-ttm.csv'
start_year = 2017
end_year = 2024
output_filename = 'datasets/ROE.csv'

roes_data = calculate_roes(income_filename, balance_filename, start_year, end_year)
roes_data.to_csv(output_filename, index=False)

Number of rows for income: 60217
Number of rows for balance: 60214
Number of rows after merge: 60205
Final number of rows: 236641
